```markdown
# Text Analytics with Azure Cognitive Services

This notebook demonstrates the use of Azure Cognitive Services for text analytics. It covers various actions such as recognizing entities, extracting Personally Identifiable Information (PII), and linking entities to external data sources. The workflow utilizes the `TextAnalyticsClient` to process sample documents and extract meaningful insights.
```

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
        TextAnalyticsClient,
        RecognizeEntitiesAction,
        RecognizeLinkedEntitiesAction,
        RecognizePiiEntitiesAction
    )
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT")
key = os.getenv("AZURE_LANGUAGE_KEY")

In [ ]:
text_analytics_client = TextAnalyticsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key),
)

In [7]:
documents = [
        'We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! '
        'They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) '
        'and he is super nice, coming out of the kitchen and greeted us all.'
        ,

        'We enjoyed very much dining in the place! '
        'The Sirloin steak I ordered was tender and juicy, and the place was impeccably clean. You can even pre-order from their '
        'online menu at www.contososteakhouse.com, call 312-555-0176 or send email to order@contososteakhouse.com! '
        'The only complaint I have is the food didn\'t come fast enough. Overall I highly recommend it!'
    ]

# Recognize Entities
This cell demonstrates how to extract general entities from the document using the `EntityRecognition` action. It retrieves information such as the entity text, category, confidence score, and offset within the document.

In [ ]:
poller = text_analytics_client.begin_analyze_actions(
        documents,
        display_name="Sample Text Analysis",
        actions=[
            RecognizeEntitiesAction()
        ])
document_results = poller.result()
for doc, action_results in zip(documents, document_results):
    print(f"\nDocument text: {doc}")
    for result in action_results:
        if result.kind == "EntityRecognition":
            print("...Results of Recognize Entities Action:")
            for entity in result.entities:
                print(f"......Entity: {entity.text}")
                print(f".........Category: {entity.category}")
                print(f".........Confidence Score: {entity.confidence_score}")
                print(f".........Offset: {entity.offset}")
    print("------------------------------------------")


Document text: We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) and he is super nice, coming out of the kitchen and greeted us all.
...Results of Recognize Entities Action:
......Entity: Contoso Steakhouse
.........Category: Location
.........Confidence Score: 0.98
.........Offset: 11
......Entity: NYC
.........Category: Location
.........Confidence Score: 1.0
.........Offset: 49
......Entity: last week
.........Category: DateTime
.........Confidence Score: 1.0
.........Offset: 53
......Entity: dinner party
.........Category: Event
.........Confidence Score: 0.77
.........Offset: 69
......Entity: food
.........Category: Product
.........Confidence Score: 0.85
.........Offset: 129
......Entity: chief cook
.........Category: PersonType
.........Confidence Score: 0.65
.........Offset: 166
......Entity: owner

# Recognize PII Entities
This cell demonstrates how to extract Personally Identifiable Information (PII) entities from the document using the `PiiEntityRecognition` action. It retrieves information such as the entity text, category, and confidence score.

In [ ]:
poller = text_analytics_client.begin_analyze_actions(
        documents,
        display_name="Sample Text Analysis",
        actions=[RecognizePiiEntitiesAction()])
document_results = poller.result()
for doc, action_results in zip(documents, document_results):
    print(f"\nDocument text: {doc}")
    for result in action_results:
        if result.kind == "PiiEntityRecognition":
            print("...Results of Recognize PII Entities action:")
            for pii_entity in result.entities:
                print(f"......Entity: {pii_entity.text}")
                print(f".........Category: {pii_entity.category}")
                print(f".........Confidence Score: {pii_entity.confidence_score}")
    print("------------------------------------------")


Document text: We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) and he is super nice, coming out of the kitchen and greeted us all.
...Results of Recognize PII Entities action:
......Entity: chief cook
.........Category: PersonType
.........Confidence Score: 0.65
......Entity: owner
.........Category: PersonType
.........Confidence Score: 0.97
......Entity: John Doe
.........Category: Person
.........Confidence Score: 0.99
------------------------------------------

Document text: We enjoyed very much dining in the place! The Sirloin steak I ordered was tender and juicy, and the place was impeccably clean. You can even pre-order from their online menu at www.contososteakhouse.com, call 312-555-0176 or send email to order@contososteakhouse.com! The only complaint I have is the food didn't come fast enough.

# Recognize Linked Entities
This cell demonstrates how to extract linked entities from the document using the `EntityLinking` action. It retrieves information such as the entity name, data source, language, entity ID, URL, and matches within the document.

In [ ]:
poller = text_analytics_client.begin_analyze_actions(
        documents,
        display_name="Sample Text Analysis",
        actions=[RecognizeLinkedEntitiesAction()])
document_results = poller.result()
for doc, action_results in zip(documents, document_results):
    print(f"\nDocument text: {doc}")
    for result in action_results:
        if result.kind == "EntityLinking":
            print("...Results of Recognize Linked Entities action:")
            for linked_entity in result.entities:
                print(f"......Entity name: {linked_entity.name}")
                print(f".........Data source: {linked_entity.data_source}")
                print(f".........Data source language: {linked_entity.language}")
                print(f".........Data source entity ID: {linked_entity.data_source_entity_id}")
                print(f".........Data source URL: {linked_entity.url}")
                print(".........Document matches:")
                for match in linked_entity.matches:
                    print(f"............Match text: {match.text}")
                    print(f"............Confidence Score: {match.confidence_score}")
                    print(f"............Offset: {match.offset}")
                    print(f"............Length: {match.length}")
    print("------------------------------------------")


Document text: We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) and he is super nice, coming out of the kitchen and greeted us all.
...Results of Recognize Linked Entities action:
......Entity name: Steakhouse
.........Data source: Wikipedia
.........Data source language: en
.........Data source entity ID: Steakhouse
.........Data source URL: https://en.wikipedia.org/wiki/Steakhouse
.........Document matches:
............Match text: Steakhouse
............Confidence Score: 0.75
............Offset: 19
............Length: 10
......Entity name: New York City
.........Data source: Wikipedia
.........Data source language: en
.........Data source entity ID: New York City
.........Data source URL: https://en.wikipedia.org/wiki/New_York_City
.........Document matches:
............Match text: NYC
............Confi